In [1]:
import pandas as pd
from pymongo import MongoClient
from datetime import datetime

# MongoDB connection setup
client = MongoClient('mongodb://localhost:27017/')
db = client['mdm_example']

def load_data(collection_name):
    collection = db[collection_name]
    data = list(collection.find({}))
    return pd.DataFrame(data)

def normalize_columns(df, id_col, name_col, email_col, **kwargs):
    column_mapping = {
        id_col: 'id',
        name_col: 'name',
        email_col: 'email'
    }
    column_mapping.update(kwargs)
    return df.rename(columns=column_mapping)

# Load and normalize data from all collections
crm_df = normalize_columns(load_data('crm_system'), 'customer_id', 'name', 'email', phone='phone')
erp_df = normalize_columns(load_data('erp_system'), 'customer_id', 'name', 'email', billing_address='address')
ecommerce_df = normalize_columns(load_data('ecommerce_system'), 'user_id', 'name', 'email', shipping_address='address')

# Combine all data into a single DataFrame
combined_df = pd.concat([crm_df, erp_df, ecommerce_df], ignore_index=True)

In [2]:

def create_golden_record_and_update_status(df):
    columns = ['id'] + [col for col in df.columns if col != 'id']
    
    golden_record = {}
    contributing_records_ids = set()
    
    for id_val, group in df.groupby('id'):
        # Create the golden record by selecting the first non-null value for each column
        record_data = {}
        for col in columns:
            if col in group.columns:
                non_null_values = group[col].dropna()
                if not non_null_values.empty:
                    record_data[col] = non_null_values.iloc[0]  # Take the first non-null value

        if not golden_record:
            golden_record = record_data
        else:
            # Update existing golden record if needed
            for col in columns:
                if col in record_data and (col not in golden_record or pd.isnull(golden_record[col])):
                    golden_record[col] = record_data[col]

        # Track contributing records' IDs
        contributing_records_ids.update(group['id'])

    # Create DataFrame for the golden record
    golden_record_df = pd.DataFrame([golden_record])
    
    # Update records that contributed to the golden record
    updated_df = df[df['id'].isin(contributing_records_ids)].copy()
    updated_df['status'] = 'end-dated'  # Or 'inactive'
    updated_df['reference_tag'] = 'Referenced'
    updated_df['end_date'] = datetime.now()

    return golden_record_df, updated_df

In [3]:
# Apply the function
golden_record_df, updated_df = create_golden_record_and_update_status(combined_df)

# Debug: Print golden_record_df info
print("Golden Record DataFrame Info:")
print(golden_record_df.info())
print("\nGolden Record Columns:")
print(golden_record_df.columns.tolist())

# Debug: Print updated_df info
print("\nUpdated DataFrame Info:")
print(updated_df.info())
print("\nUpdated DataFrame Columns:")
print(updated_df.columns.tolist())

Golden Record DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  1 non-null      object 
 1   _id                 1 non-null      object 
 2   name                1 non-null      object 
 3   email               1 non-null      object 
 4   phone               1 non-null      object 
 5   billing_address     1 non-null      object 
 6   credit_limit        1 non-null      float64
 7   address             1 non-null      object 
 8   last_purchase_date  1 non-null      object 
 9   shipping_address    1 non-null      object 
dtypes: float64(1), object(9)
memory usage: 212.0+ bytes
None

Golden Record Columns:
['id', '_id', 'name', 'email', 'phone', 'billing_address', 'credit_limit', 'address', 'last_purchase_date', 'shipping_address']

Updated DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Rang

In [4]:
def save_to_mongodb(df, collection_name):
    if not df.empty:
        collection = db[collection_name]
        collection.delete_many({})  # Clear existing records
        collection.insert_many(df.to_dict('records'))
        print(f"Data saved to collection: {collection_name}")
    else:
        print(f"No data to save for collection: {collection_name}")

# Save golden record and updated records back to MongoDB
save_to_mongodb(golden_record_df, 'Golden3')  # Save to 'Golden2' collection
save_to_mongodb(updated_df, 'mdm_example_updated')  # Save the updated DataFrame back to MongoDB

Data saved to collection: Golden3
Data saved to collection: mdm_example_updated


In [5]:
print("\nGolden Record DataFrame:")
print(golden_record_df)

print("\nUpdated DataFrame with statuses:")
print(updated_df)

print("Process completed successfully.")


Golden Record DataFrame:
     id                       _id        name                 email  \
0  C001  66cedd48983b8bf3f5406b79  John Smith  john.smith@email.com   

          phone             billing_address  credit_limit  \
0  123-456-7890  123 Main St, City1, State1        5000.0   

                      address last_purchase_date  \
0  123 Main St, City1, State1         2023-02-20   

                shipping_address  
0  456 Oak Avenue, City2, State2  

Updated DataFrame with statuses:
                         _id       id            name  \
0   66cedd48983b8bf3f5406b79     C001      John Smith   
1   66cedd48983b8bf3f5406b7a     C002        Jane Doe   
2   66cedd48983b8bf3f5406b7b     C003    Mike Johnson   
3   66cedd48983b8bf3f5406b7c     C004     Emily Brown   
4   66cedd48983b8bf3f5406b7d     C005       David Lee   
5   66cedd48983b8bf3f5406b7e     C006    Sarah Wilson   
6   66cedd48983b8bf3f5406b7f     C007    Chris Taylor   
7   66cedd48983b8bf3f5406b80     C008    Am